In [1]:
import os 

import torch
import torch.nn as nn 
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from helpers import load_experiment

In [2]:
# baseline /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/18-14-45
# framework (simple) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/19-47-58
# framework /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/20-17-31
# framework (parallel) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/21-18-11
# baseline (sigmoid) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-12/06-58-58

dm, model = load_experiment("/home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/20-17-31")

Global seed set to 42


Fetching configuration...
Loading datamodule...


[nltk_data] Downloading package wordnet to /home/danis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 2700/2700 [00:00<00:00, 8550.22it/s]


Len of vocab:  54
Max len of caption:  16
Index for <pad>: [0]
Loading model


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_task' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion_task'])`.
  rank_zero_warn(
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_tie' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion_tie'])`.
  rank_zero_warn(


In [3]:
train_loader = dm.train_dataloader()
train_set = train_loader.dataset

In [4]:
# [red, green, blue, square, triangle, circle]

attribute_mapping = {
    0: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0
        ],
    1: [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
        ],
    2: [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0
        ],
    3: [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0
        ],
    4: [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0
        ],
    5: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0
        ],
    6: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0
        ],
    7: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0
        ],
    8: [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1
        ],
}

In [5]:
batch = next(iter(train_loader))

In [11]:
img_ = batch["image"][0].cuda().unsqueeze(0)
ind_ = batch["indices"][0].cuda().unsqueeze(0)

a = model(img_, ind_)

a["feature_activated"], a["concept_activated"]

tensor(0.0116, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.2128, device='cuda:0', grad_fn=<MaxBackward1>)
Parameter containing:
tensor(1.0729, device='cuda:0', requires_grad=True) Parameter containing:
tensor(0.3637, device='cuda:0', requires_grad=True)
(tensor(-6.0242, device='cuda:0', grad_fn=<MinBackward1>), tensor(-2.5062, device='cuda:0', grad_fn=<MinBackward1>)) (tensor(1.3049, device='cuda:0', grad_fn=<MaxBackward1>), tensor(1.0507, device='cuda:0', grad_fn=<MaxBackward1>)) (tensor(0.5420, device='cuda:0', grad_fn=<MinBackward1>), tensor(0.2299, device='cuda:0', grad_fn=<MinBackward1>))


(tensor([[0., 0., 0., 1., 0., 1., 0., 0., 1., 1.]], device='cuda:0',
        grad_fn=<AddBackward0>),
 tensor([[1., 0., 0., 1., 0., 0., 0., 1., 1., 1.]], device='cuda:0',
        grad_fn=<AddBackward0>))

In [12]:
a

{'feature_logits': tensor([[-0.9177, -2.9081, -6.0242,  1.3049, -3.9880, -1.8043, -2.5249,  0.5420,
           0.6947, -2.7098]], device='cuda:0', grad_fn=<AddmmBackward0>),
 'concept_logits': tensor([[ 0.4346, -2.1049, -0.9460,  1.0507, -2.5062, -0.5579, -2.4468,  0.2299,
           0.7544, -2.4062]], device='cuda:0', grad_fn=<SqueezeBackward1>),
 'feature_probs': tensor([[0.2854, 0.0518, 0.0024, 0.7867, 0.0182, 0.1413, 0.0741, 0.6323, 0.6670,
          0.0624]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 'concept_probs': tensor([[0.6070, 0.1086, 0.2797, 0.7409, 0.0754, 0.3640, 0.0797, 0.5572, 0.6801,
          0.0827]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 'feature_activated': tensor([[0., 0., 0., 1., 0., 1., 0., 0., 1., 1.]], device='cuda:0',
        grad_fn=<AddBackward0>),
 'concept_activated': tensor([[1., 0., 0., 1., 0., 0., 0., 1., 1., 1.]], device='cuda:0',
        grad_fn=<AddBackward0>),
 'prediction': tensor([[ 0.0444, -1.8798,  0.0704,  1.5357, -1.8278, -0.81

In [14]:
sample = train_set[50]
image = sample["image"]
captions = sample["report"]

image = image.unsqueeze(0).cuda()
captions = captions.unsqueeze(0).cuda()


model.main.inference(image)[1].cpu().detach().numpy()[0]
# model(image)["concept_probs"].cpu().detach().numpy()[0]

AttributeError: 'str' object has no attribute 'unsqueeze'

In [13]:
model(image)

TypeError: LitAutoConceptBottleneckModel.forward() missing 1 required positional argument: 'indices'

In [6]:
model(image, torch.tensor([[0, 10, 1]]).cuda())

tensor(0.0303, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.2492, device='cuda:0', grad_fn=<MaxBackward1>)
Parameter containing:
tensor(1.0729, device='cuda:0', requires_grad=True) Parameter containing:
tensor(0.3637, device='cuda:0', requires_grad=True)


{'feature_logits': tensor([[ 1.1863, -4.5293,  1.4814, -2.2510, -4.0154,  1.1285,  1.5703, -0.6106,
           1.1538, -2.9983]], device='cuda:0', grad_fn=<AddmmBackward0>),
 'concept_logits': tensor([[-0.2064, -0.3806, -0.6131, -0.3321,  0.9779, -0.7432, -0.4473,  0.9430,
           1.1664, -2.3901]], device='cuda:0', grad_fn=<SqueezeBackward1>),
 'feature_probs': tensor([[0.7661, 0.0107, 0.8148, 0.0953, 0.0177, 0.7556, 0.8278, 0.3519, 0.7602,
          0.0475]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 'concept_probs': tensor([[0.4486, 0.4060, 0.3513, 0.4177, 0.7267, 0.3223, 0.3900, 0.7197, 0.7625,
          0.0839]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 'feature_activated': tensor([[1., 0., 1., 0., 0., 1., 1., 1., 1., 0.]], device='cuda:0',
        grad_fn=<AddBackward0>),
 'prediction': tensor([[ 2.4578,  0.1548,  0.3244, -0.5086, -0.1632, -1.1643,  0.1855, -0.4218,
          -0.6592]], device='cuda:0', grad_fn=<NativeBatchNormBackward0>),
 'scores': tensor([[[1.382

In [7]:
scores = list()

for feature_id in range(10):

    attribute_values = [[0, 0] for _ in range(len(attribute_mapping[0]))]

    for i in tqdm(range(len(train_set))):
        sample = train_set[i]
        
        idx = sample["target"].item()
        attributes = np.array(attribute_mapping[idx])

        image = sample["image"]
        image = image.unsqueeze(0).cuda()

        feature = model.main.inference(image)[1].cpu().detach().numpy()[0][feature_id]
        # feature = model(image)["concept_probs"].cpu().detach().numpy()[0][feature_id]

        for attr_id, attribute in enumerate(attributes):
            value_on = attribute * feature + (1 - attribute) * (1 - feature)
            attribute_values[attr_id][0] += value_on

            value_off = attribute * (1 - feature) + (1 - attribute) * feature
            attribute_values[attr_id][1] += value_off
    
    attribute_values_max = [max(p) for p in attribute_values]
    attribute_values_max = max(attribute_values_max)
    score = attribute_values_max / len(train_set)

    scores.append(score)

metric = np.array(scores).mean()

print(metric)

100%|██████████| 1845/1845 [00:40<00:00, 45.92it/s]

0.7172063330401222


Baseline: 0.7722590434662505

Framework (simplify): 0.7840587777528067

Framework: 0.7172063330401222

Baseline: 0.7481238945043126 / 0.8002787673469156 / 

Our framework: 0.733698356318566 / 0.733698356318566 / 0.77456 (w/o textual, anneal)


In [56]:
scores, np.array(scores).mean() # baseline

([0.8573726896832234,
  0.839452540636915,
  0.7797192060542814,
  0.8681423352541888,
  0.8260357244465,
  0.8895389913930329,
  0.8637059218046688,
  0.7592228388927755,
  0.797043976947101,
  0.7546556928753102],
 0.8234889917987998)

In [9]:
scores # framework

[0.7648627196583573,
 0.6768948672750362,
 0.7658918708248464,
 0.9917780249794046,
 0.8336742381045732,
 0.7635008184147333,
 0.7613273321342036,
 0.7518359585674752,
 0.6415101127414388,
 0.7674385555471499]

tensor([[ 0.0122, -0.0529, -0.0110,  ..., -0.0344, -0.0533,  0.0093],
        [ 0.0010,  0.0120,  0.0526,  ..., -0.0378,  0.0056, -0.0115],
        [-0.0123,  0.0306,  0.0034,  ..., -0.0043, -0.0447,  0.0298],
        ...,
        [ 0.0362,  0.0370, -0.0468,  ...,  0.0215,  0.0100,  0.0521],
        [ 0.0442,  0.0317, -0.0362,  ...,  0.0499, -0.0278,  0.0430],
        [ 0.0372, -0.0121,  0.0365,  ..., -0.0190, -0.0329, -0.0160]],

Predictor:  Parameter containing:
tensor([[-0.0137, -0.0356,  0.0175,  ..., -0.0315,  0.0064, -0.0237],
        [-0.0178,  0.0340,  0.0079,  ...,  0.0032, -0.0259, -0.0187],
        [ 0.0283, -0.0175, -0.0104,  ..., -0.0227, -0.0323, -0.0349],
        ...,
        [-0.0439,  0.0097, -0.0294,  ..., -0.0208,  0.0355,  0.0253],
        [-0.0421, -0.0303, -0.0202,  ..., -0.0343,  0.0074,  0.0075],
        [ 0.0195,  0.0237, -0.0137,  ..., -0.0427, -0.0188,  0.0273]],

In [16]:
import torch


isinstance(torch.nn.Linear(10, 5), torch.nn.Linear)

True